# Overview 

In the first ~1/3 of this lecture, we will review homework, programming, and fMRI concepts

In the second ~1/3 of this lecture, we will introduce data normalization and logical indexing

In the last ~1/3 of this lecture we will introduce software to display fMRI data on a 3D representation of the cortical surface.

# Goals
* Understand the way pycortex represents a volumetric data set
* Create pycortex Volume objects from 3D data, masked data, and time series data
* Display data on the cortical surface



# Updating resources in your server home directory
(Run the cells in this section once, then restart your kernel, reload the web page, and skip this section the next time through!)

In [ ]:
if False: # set this to True if you still need to run this, but it should only be run once.
    # Updating functions
    import neurods
    import cortex as cx
    # Add figures to notebook
    dropbox_link = 'https://www.dropbox.com/s/dkibicpvc13ng27/Archive.zip'
    fname = 'figures/'
    basedir = '~/cogneuro-connector/weekly_materials/Lecture04_Normalization_Masking_pycortex/'
    neurods.io.download_file(dropbox_link, fname, basedir, zipfile=True)
    # Update neurods package
    neurods.io.update_neurods()
    # Update pycortex configuration file
    cx.config.set('basic', 'filestore', '/data/shared/cogneuro88/pycortex_store/')
    with open(cx.options.usercfg, 'w') as fid:
        cx.config.write(fid)

# Part 1:  Writing reusable functions
& misc. code / efficiency tips!

In [ ]:
# Load some necessary libraries
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
# Set plotting defaults
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

We created the following functions to make plots of slices of fMRI volumes. In some code libraries, these are called light table or mosaic plots, because in the (literally) dark old days of film photography, photographers used to lay out their film negatives on light tables in a format similar to this to see them.

We will want to make plots like this many times throughout the course, so we would like to formalize these functions a little more to make them more readily reusable. 

In [ ]:
def get_any_slice(volume, slice_number, dimension):
    """Given an integer and a 3D volume, this function returns the data of 
    that horizontal slice """ 
    if dimension == 0:
        img = volume[slice_number, :, :]
    elif dimension == 1:
        img = volume[:, slice_number, :]
    elif dimension == 2:
        img = volume[:, :, slice_number]
    return img

def plot_any_slice_v2(volume, slice_number, dimension, cmap = 'viridis', vmin=0, vmax=2000,
                     origin = 'lower', interpolation='nearest', aspect='equal'):
    img = get_any_slice( volume, slice_number, dimension)
    _ = plt.imshow(img, cmap = cmap, vmin= vmin, vmax = vmax, origin = origin, interpolation = interpolation,
                  aspect = aspect)
    _ = plt.axis('off')
    
def plot_all_slices(volume, slice_dimension, nrows, ncols , cmap = 'viridis', vmin=0, vmax=2000,
                     origin = 'lower', interpolation='nearest', aspect='equal' ):
    fig = plt.figure(figsize = (8,8))
    n_slices = volume.shape[slice_dimension]
    for s in range(n_slices):
        ax = fig.add_subplot(nrows, ncols, s+1)
        plot_any_slice_v2(first_volume, s, slice_dimension, cmap = cmap, vmin= vmin, vmax = vmax, 
                          origin = origin, interpolation = interpolation,aspect = aspect)    

We will modify the final function to be more production-ready below. Follow along...

* One major addition we need to make is a ***docstring***. See [here](https://github.com/numpy/numpy/blob/master/doc/HOWTO_DOCUMENT.rst.txt#docstring-standard) for a general description of writing docstrings according to the numpy format, and [here](https://github.com/numpy/numpy/blob/master/doc/example.py) for clear examples.
* We will use the `**kwargs` syntax to pass keyword arguments from function to function
* We will modify argument names to be more similar to arguments in similar functions

**NOTE** I forgot to mention this during class, but there is also a way to set global defaults for plots in matplotlib. Once you have called

    import matplotlib.pyplot as plt

you can call the following cell to set defaults for the `imshow()` function:

In [ ]:
### STUDENT ANSWER
plt.rcParams['image.cmap'] = 'viridis'
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.origin'] = 'lower'
plt.rcParams['image.aspect'] = 'equal'

In [ ]:
### STUDENT ANSWER
def get_any_slice(volume, slice_number, dimension):
    """Given an integer and a 3D volume, this function returns the data of 
    that horizontal slice """ 
    if dimension == 0:
        img = volume[slice_number, :, :]
    elif dimension == 1:
        img = volume[:, slice_number, :]
    elif dimension == 2:
        img = volume[:, :, slice_number]
    return img

def slice_3d_array(volume, axis, nrows, ncols , vmin=0, vmax=2000, 
                   figsize=(8,8), slice_prefix=None, **kwargs):
    """Plot slices of 3D array along an arbitrary axis
    
    Parameters
    ----------
    volume : array
        3D array to be plotted
    axis : int
        dimension to be sliced / plotted
    nrows, ncols : int
        number of rows / columns in plot
    vmin, vmax : scalar
        miniumum / maximum value for color scale all plots
    figsize : tuple
        (X, Y) size in inches for figure
    slice_prefix : string or None
        If provided as a string, this is a format string for titles of each
        individual plot (titles will be `slice_prefix.format(slice_number)`)
        if None, no titles are added
    Other Parameters
    ----------------
    kwargs : named keyword arguments
        keyword arguments are passed to imshow()
    
    """
    fig = plt.figure(figsize = figsize)
    n_slices = volume.shape[axis]
    for s in range(n_slices):
        ax = fig.add_subplot(nrows, ncols, s+1)
        img = get_any_slice(volume, s, axis)
        ax.imshow(img, vmin = vmin, vmax = vmax, **kwargs)
        if slice_prefix is not None:
            ax.set_title(slice_prefix.format(s))
        ax.set_axis_off()
    plt.tight_layout()

In [ ]:
# You can view the docstring we just wrote by calling:
slice_3d_array?

In [ ]:
# Test the function with some random data:
rdata = np.random.uniform(size=(30,100,100))
# (Note that if you don't set vmax here, the plots look blank!)
slice_3d_array(rdata, 0, 6, 5, vmax=1, figsize=(5,6))

Since we will be needing this function (or one very like it) throughout the semester, we have incorporated the function (with a few extra bells and whistles) into our `neurods` package. This is a useful workflow: figure out the rudiments of what you want from a function in notebooks or otherwise at the command line, incorporate it into a more permanent library, and continue to make changes to it until it performs its one function very well. 

In [ ]:
# Import neurods package
import neurods

In [ ]:
# Show where the text files supporting this package are on this machine
print(neurods.__file__)

In [ ]:
# Show help (docstring) for the function
neurods.viz.slice_3d_array?

In [ ]:
# Show full text of the function (note that there are a few differences between this and the 
# one we wrote in class above)
neurods.viz.slice_3d_array??

In [ ]:
# Make use of that function
fig = plt.figure(figsize=(6,5))
_ = neurods.viz.slice_3d_array(rdata, axis=0, fig=fig)

## Survey 1!

Asked about shapes of new data resulting from different ways to index into a data set. 

i.e., if a variable `data` is of shape (120, 25, 104, 104), what is the shape of: 
* `data[12]`
* `data[:10, 0, :, :]`
* `data[:, :1, :, :]`
* `data.T[..., 12]`

(if you missed your chance to answer the question in class, see if you can predict what shape each will be before creating and checking your answer!)

You can check the answers by, e.g. :

    data = np.random.rand(120, 25, 104, 104)
    print(data[12].shape)

# Load data
Today, we will load data from a common neuroimaging data format ([NIfTI](https://nifti.nimh.nih.gov/nifti-1/) format). If you find open source neuroimaging data online, it will most likely be in this format. To load this data, we need to use a neuroimaging code library called nibabel. 

In [ ]:
import nibabel
import os

This is where data for class will be stored (this directory will be updated with more data throughout the semester).

In [ ]:
ls /data/shared/cogneuro88/fMRI/

In [ ]:
# Create a nifti (nii) proxy object
fbase = '/Users/mark/Dropbox/data8/' #'/data/shared/cogneuro88'
fname = os.path.join(fbase,'fMRI/categories/s01_categories_01.nii.gz')
nii = nibabel.load(fname)

# This object stores the infomation *about* the fMRI data stored in the file. 
# This meta-data can be accessed via attributes of the `nii` object.
print('nii.in_memory :', nii.in_memory)
print('nii.shape :', nii.shape)
print('voxel sizes :', nii.header.get_zooms())

There is also information stored about how the brain was oriented in space as it was scanned, but that is beyond the scope of what we will go into here. 

In [ ]:
# Retrieve actual data as an array
data = nii.get_data().T
print('nii.in_memory : ', nii.in_memory)
print('data shape : ', data.shape)

In [ ]:
# Plot a few voxels
_ = plt.plot(data[:,22:, 45, 45])
_ = plt.xlabel('Time (TRs)')
_ = plt.ylabel('BOLD signal')

### HW Recap

In [ ]:
# Plot an image of the first 10 time points for all voxels in one horizontal slice
slice_3d_array(data[:10,5], 0, 2, 5, slice_prefix='Time = {}', figsize=(10,4), vmax=1000)

Note that VERY LITTLE appears to change across these images, because the differences in signal between voxels are greater than the differences across time! This makes visualizations of our data difficult / hard to interpret. Let's look at this a different way:

### BREAKOUT SESSION
Make an image plot of all the voxels in one horizontal slice. 

1. Select all time points for one horizontal slice
2. Use np.reshape make a 2D array that is (time x voxels) 
3. Use `plt.imshow` to show that 2D array
4. Make the plot pretty! (label axes, set an appropriate color scale, etc)

In [ ]:
# remember to check on a function's help before trying to use it!
np.reshape?

In [ ]:
# Also useful for clarifying the form of lesser-used keyword arguments...
plt.imshow?

In [ ]:
### STUDENT ANSWER
plt.figure(figsize=(10,4))
horiz = data[:, 5].reshape(120, 100*100)
plt.imshow(horiz, aspect='auto')
plt.xlabel('Voxels')
plt.ylabel('Time (TRs)');

The plot above shows clearly that the problem is that there's more variability across voxels than there is across time (Some voxels *always* show higher signal than others). This scaling issue complicates some kinds of visualization and analysis. To look at this issue in yet another way, let's look at histograms of the timecourses for two voxels.

# Part 2: Data normalization

In [ ]:
# (New function in next cell - check to see what it does!)
np.linspace?

In [ ]:
bins = np.linspace(250, 650, 31)
voxel_a = data[:, 5, 50, 43]
voxel_b = data[:, 5, 50, 53]
_ = plt.hist(voxel_a, bins, label='Voxel A')
_ = plt.hist(voxel_b, bins, label='Voxel B')
plt.legend(frameon=False) # frameon=False -> New fanciness
# Add labels! (this was the topic of survey 2)
_ = plt.xlabel('Raw fMRI activity\n(Arbitrary units)', fontsize=16)
_ = plt.ylabel('Count (TRs for which\nsignal fell into one bin)', fontsize=16)

## Survey 2!

In a previous homework, we normalized data by setting the minimum of the data to 0 and the max to 1, like this:

In [ ]:
def minmax_norm(data):
    """Normalize data to range of 0-1 by subtracting min, dividing by range"""
    data_norm = (data-data.min()) / (data.max()-data.min())
    return data_norm

In [ ]:
print((np.min(minmax_norm(voxel_a)), np.max(minmax_norm(voxel_b))))

Show the results of normalizing the data this way:

In [ ]:
bins01 = np.linspace(0,1,31)
voxel_a_n = minmax_norm(voxel_a)
voxel_b_n = minmax_norm(voxel_b)
_ = plt.hist(voxel_a_n, bins01, label='Voxel A')
_ = plt.hist(voxel_b_n, bins01, label='Voxel B')
_ = plt.legend(frameon=False)

Looks sensible. However, a problem with this normalization method shows up when you have outlying values. What would happen if Voxel A, at one time point, had a (spuriously) very large value? Let's see!

In [ ]:
# First, an aside: a cautionary tale. Python memory is weird.
a = np.arange(10)
b = a
b[3] = 63
print(a)

In [ ]:
# You can avoid changing a variable by using the copy package
import copy
a = np.arange(10)
b = copy.copy(a)
b[3] = 63
print(a)

In [ ]:
# Create a variable called voxel_a_wonky and add an outlier to it
voxel_a_wonky = copy.copy(voxel_a)
voxel_a_wonky[5] = voxel_a_wonky.max()*1.5

In [ ]:
# Show what you did to the data
plt.plot(voxel_a_wonky)
_ = plt.xlabel('Time')
_ = plt.ylabel('Raw fMRI Activity')

In [ ]:
# See what that outlier does to histogram plots created after our first normalization method!
_ = plt.hist(minmax_norm(voxel_a_wonky), bins01, label='Voxel A (w/ outlier)')
#_ = plt.hist(minmax_norm(voxel_b), bins01, label='Voxel B') # uncomment this to show voxel_b
plt.legend()

This does not put the data from both voxels into a similar range, because the max value is not stable (it can change a lot depending on only one data point). In general, (linear) data normalization set you need to *center* it (by subtracting off some value) and to *scale* it (by dividing by some value or performing some nonlinear operation). 
A more robust, stable way to normalize data is to subtract the *mean* of the data instead of the min, and to divide by the *standard deviation* instead of the range. 

The *standard deviation* is a measure of the variability of the data, derived from the whole data set (rather than the two points - the min and the max - that we used before.)

This process converts the data to *standard scores* or *z scores*. 

In [ ]:
# You can compute the standard deviation of voxel A and voxel B using 
print(np.std(voxel_a), ',', np.std(voxel_b))
# or: 
print(voxel_a.std(), ',', voxel_b.std())

In [ ]:
# We can look at the standard deviation of the data by slice
fig = plt.figure(figsize=(6,5))
_ = neurods.viz.slice_3d_array(data.std(axis=0), axis=0, fig=fig)

The plot above shows the places in the brain where the signal was most variable!

### Breakout session
Perform normalization of Voxel A and Voxel B by (1) subtracting off the mean (**`array.mean()`** or **`np.mean(array)`**) and (2) dividing by the standard deviation (**`array.std()`** or **`np.std(array)`**)

Then repeat the plots we created above (making histograms of the timecourses for Voxel A, with and without an outlier, and for Voxel B)

In [ ]:
### STUDENT ANSWER
def znorm(data):
    data_norm = (data-data.mean()) / data.std()
    return data_norm

# Note the bins! 
#bins = 21 # In the first histogram plot of data that you make for a given data set, 
          # you may not want to specify bins (in case you miss data outside the 
          # range of bins you specify)
bins = np.linspace(-3,3,31)
# alpha values make plot elements transparent 
# (0 = invisible, 0.5 = partly see-through, 1.0 = opaque)
_ = plt.hist(znorm(voxel_a), bins, label='Voxel A', color='b', alpha=.5)
_ = plt.hist(znorm(voxel_a_wonky), bins, label='Voxel A, w/ outlier', color='y', alpha=.5)
#_ = plt.hist(znorm(voxel_b), bins01, label='Voxel B', color='r', alpha=0.5)
_ = plt.legend(frameon=False)
_ = plt.xlabel('Activity (z-normalized)')
_ = plt.ylabel('Count (TRs)')

z normalization can be performed easily on multi-dimensional arrays using the `zscore` function in scipy.stats. 

In [ ]:
from scipy.stats import zscore
zscore?

In [ ]:
data_z = zscore(data, axis=0)

Now we can repeat a few of the plots we made above, to show that we now see more variation across time than across voxels.

In [ ]:
# Plot time x voxels
plt.figure(figsize=(10,4))
plt.imshow(data_z[:, 5].reshape(120, 100*100), aspect='auto', vmin=-3, vmax=3)
plt.xlabel('Voxels')
plt.ylabel('Time (TRs)')

In [ ]:
# Plot 
slice_3d_array(data_z[:10,5], 0, 2, 5, figsize=(10,4), slice_prefix='T = {}', vmin=-3, vmax=3)

# Part 3: (Skipped - we will do this next week!)

# Onward to 3D data visualizations! 

We will use a python module called **`pycortex`** to show data in 3D on the brain. This module was developed here at UC Berkeley in the Gallant lab, mostly by James Gao, with help from Alex Huth, Mark Lescroart, and other lab members. The code is freely available online [here](https://github.com/gallantlab/pycortex), and a paper summarizing the code can be found [here](http://journal.frontiersin.org/article/10.3389/fninf.2015.00023/full). 

To map the functional data onto the cortex, pycortex requires at least two things:

1. The cortical surface of the subject. 
    * pycortex stores cortical surface files (and several other files) for each subject in a reliably structured directory of files. Because of this reliable directory structure, all we need to provide to the code is a subject ID string, and the code will be able to find and load the relevant cortical surface files. 
2. The functional to anatomical aligmnent of this data to that cortical surface
    * Alignment of functional data to anatomical data proceeds by an *affine transform*. How this transformation works is beyond the scope of this class, but you can look it up on [wikipedia](https://en.wikipedia.org/wiki/Affine_transformation) or in your favorite linear algebra textbook if you're curious. The practical upshot is that a 4x4 matrix of numbers is sufficient to store the 3 rotations (around the x, y, and z axes) and 3 the transformations (in the x, y and z dimensions) that will transform the functional data in space such that they are aligned with the anatomical data (with the cortical surface). In the pycortex code (and in other neuroimaging software), "transform" is abbreviated in variable names as `xfm`. Just as with the cortical surface, we only need to specify a name for a transform, and the code will know where to find the file that contains the affine transformation matrix. 

In [ ]:
import cortex

In [ ]:
# (1) subject (specifies the cortical surface of the brain)
subject = 's01' 
# (2) transform = functional-to-anatomical alignment
transform = 'catloc' 

In [ ]:
# Create a volume
data_volume = cortex.Volume(data[0], subject, transform) 

Click the link that this generates to see the webgl-based viewer. You can click and drag the brain around to view it from different angles. If you move the mouse up to the gray  top of the browser window, a menu will come down with more display options. If you move the mouse to the bottom of the browser window, there are sliders to adjust the amount of unfolding of the brain and other view parameters. There are also hotkeys you can use to inflate, flatten, and re-fold the brain in the main window (press "f" to flatten, "i" to inflate, and "r" to re-fold"). 

In [ ]:
# Show the volume in a 3D brain (ooooh)
cortex.webgl.show(data_volume)

In [ ]:
# Make an image plot of a flatmap of the cortex
cortex.quickflat.make_figure(data_volume)